In [ ]:
import os

import gymnasium as gym
from gymnasium import spaces
from stable_baselines3 import PPO
from stable_baselines3.common.env_checker import check_env

import heapq
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.animation as animation
from matplotlib.animation import FuncAnimation
from IPython.display import display, clear_output

In [ ]:
class DGSN_Env(gym.Env):
    def __init__(self, grid_size=10, max_static_obstacles=10, max_dynamic_obstacles=6, max_steps=100000):
        super(DGSN_Env, self).__init__()
        self.grid_size = grid_size
        self.max_static_obstacles = max_static_obstacles
        self.max_dynamic_obstacles = max_dynamic_obstacles
        self.num_static_obstacles = np.random.randint(1,10)
        self.num_dynamic_obstacles = np.random.randint(1,7)
        self.agent_pos = np.array(np.random.uniform(0, self.grid_size, size=2).round(1))
        self.goal_pos = np.array(np.random.uniform(0, self.grid_size, size=2).round(1))
        self.dynamic_obstacles = self._init_dynamic_obstacles()
        self.static_obstacles = self._init_static_obstacles()
        
        self.dist_factor = 200
        self.max_steps = max_steps
        self.current_step = 0
        
        self.action_space = spaces.Discrete(9)
        self.observation_space = spaces.Dict({
            'agent': spaces.Box(low=0, high=grid_size, shape=(2,), dtype=np.float32),
            'dyn_obs': spaces.Box(low=0, high=grid_size, shape=(self.max_dynamic_obstacles, 2), dtype=np.float32),
            'sta_obs': spaces.Box(low=0, high=grid_size, shape=(self.max_static_obstacles, 2), dtype=np.float32)
        })
        
        print("*******************************************************************\n")
        print("Initialized the environment with the following")
        print("Agent's Initial Position :", self.agent_pos)
        print("Goal Position :", self.goal_pos)
        print("Number of Static Obstacles :", self.num_static_obstacles)
        print("Static Obstacle Positions :", self. static_obstacles)
        print("Number of Dynamic Obstacles :", self.num_dynamic_obstacles)
        print("Dynamic Obstacle Positions :", self. dynamic_obstacles, "\n")
        print("*******************************************************************")
        
    def _init_static_obstacles(self):
        obstacles = []
        for _ in range(self.num_static_obstacles):
            obstacles.append(np.random.uniform(0, self.grid_size, size=2).round(1))
        return obstacles
    
    def _init_dynamic_obstacles(self):
        obstacles = []
        for _ in range(self.num_dynamic_obstacles):
            start_pos = np.random.uniform(0, self.grid_size, size=2).round(1)
            end_pos = np.random.uniform(0, self.grid_size, size=2).round(1)
            path = self._compute_path(start_pos, end_pos)
            obstacles.append({
                'start': start_pos,
                'end': end_pos,
                'current': start_pos.copy(),
                'angle': np.rad2deg(np.arctan2(end_pos[1] - start_pos[1], end_pos[0] - start_pos[0])),
                'distance': np.linalg.norm(end_pos - start_pos),
                'path': path if path else []
            })
        return obstacles
    
    def _compute_path(self, start, end):
        def heuristic(a, b):
            return np.linalg.norm(a - b)

        def a_star(start, goal):
            open_set = []
            heapq.heappush(open_set, (0, tuple(start)))
            came_from = {}
            g_score = {tuple(start): 0}
            f_score = {tuple(start): heuristic(start, goal)}

            while open_set:
                _, current = heapq.heappop(open_set)
                current = np.array(current)

                if np.array_equal(current, goal):
                    path = []
                    while tuple(current) in came_from:
                        path.append(current)
                        current = came_from[tuple(current)]
                    path.append(start)
                    path.reverse()
                    return path

                neighbors = [current + [0.1, 0], current + [-0.1, 0], current + [0, 0.1], current + [0, -0.1]]
                neighbors = [np.clip(neighbor, 0, self.grid_size - 0.5) for neighbor in neighbors]

                for neighbor in neighbors:
                    tentative_g_score = g_score[tuple(current)] + heuristic(current, neighbor)
                    if tuple(neighbor) not in g_score or tentative_g_score < g_score[tuple(neighbor)]:
                        came_from[tuple(neighbor)] = current
                        g_score[tuple(neighbor)] = tentative_g_score
                        f_score[tuple(neighbor)] = tentative_g_score + heuristic(neighbor, goal)
                        heapq.heappush(open_set, (f_score[tuple(neighbor)], tuple(neighbor)))

            return []

        return a_star(start, end)
    
    def step(self, action):
        self.current_step += 1
        previous_agent_pos = self.agent_pos.copy()
        print(f"Agent's Current Position {previous_agent_pos}")
        
        if action == 0:
            self.agent_pos[1] += 0.1
            print("Action Taken : Up")
        elif action == 1:
            self.agent_pos[1] -= 0.1
            print("Action Taken : Down")
        elif action == 2:
            self.agent_pos[0] -= 0.1
            print("Action Taken : Left")
        elif action == 3:
            self.agent_pos[0] += 0.1
            print("Action Taken : Right")
        elif action == 4:
            self.agent_pos += [0.1, 0.1]
            print("Action Taken : Right-Up")
        elif action == 5:
            self.agent_pos += [-0.1, 0.1]
            print("Action Taken : Left-Up")
        elif action == 6:
            self.agent_pos += [0.1, -0.1]
            print("Action Taken : Right-Down")
        elif action == 7:
            self.agent_pos += [-0.1, -0.1]
            print("Action Taken : Left-Down")
        elif action == 8:
            print("Action Taken : Stay Still!")
            pass
        
        self.agent_pos = np.clip(self.agent_pos, 0, self.grid_size - 0.5)
        print(f"Agent's New Position {self.agent_pos}")
        
        for obstacle in self.dynamic_obstacles:
            if len(obstacle['path']) > 1:
                obstacle['current'] = obstacle['path'].pop(0)
            else:
                obstacle['path'] = self._compute_path(obstacle['start'], obstacle['end'])
        
        reward = self._compute_reward(previous_agent_pos)
        print(f"Reward Obtain : {reward}")
        done = self._is_done()
        if self.current_step >= self.max_steps:
            done = True
            reward -= 50
        truncated = False
        
        return self._get_obs(), reward, done, truncated, {}
    
    def _compute_reward(self, previous_agent_pos):
        previous_dist_to_goal = np.linalg.norm(previous_agent_pos - self.goal_pos)
        current_dist_to_goal = np.linalg.norm(self.agent_pos - self.goal_pos)
        reward = (current_dist_to_goal - previous_dist_to_goal) * self.dist_factor
        
        # Check for collisions with obstacles and boundaries
        for obs in self.dynamic_obstacles:
            angle = np.rad2deg(np.arctan2(obs['current'][1] - self.agent_pos[1], obs['current'][0] - self.agent_pos[0]))
            distance = np.linalg.norm(obs['current'] - self.agent_pos)
            # Update reward based on obstacle proximity
            # reward -= distance*self.dist_factor  #
        if distance < 0.5:  # Define a threshold for collision
            reward -= 10*(1/distance)  # Penalty for colliding with a dynamic obstacle
        if distance == 0:
            reward -= 125
        if any(np.array_equal(self.agent_pos, obs) for obs in self.dynamic_obstacles):
            reward -= 15  # Penalty for colliding with a Dynamic obstacle
        if any(np.array_equal(self.agent_pos, obs) for obs in self.static_obstacles):
            reward -= 15  # Penalty for colliding with a Static obstacle
        if np.any(self.agent_pos == 0) or np.any(self.agent_pos >= self.grid_size - 0.5):
            reward -= 5  # Penalty for hitting the wall/boundary
            
        reward -= 1
        return reward
    
    def _is_done(self):
        return np.array_equal(self.agent_pos, self.goal_pos)
    
    def _get_obs(self):
        agent_state = np.array([self.agent_pos[0], self.agent_pos[1]], dtype=np.float32)
        dynamic_obstacle_states = np.array([(ob['angle'], ob['distance']) for ob in self.dynamic_obstacles] + 
                                           [np.zeros(2) for _ in range(self.max_dynamic_obstacles - len(self.dynamic_obstacles))],
                                           dtype=np.float32)
        static_obstacle_states = np.array([ob[:2] for ob in self.static_obstacles] + 
                                          [np.zeros(2) for _ in range(self.max_static_obstacles - len(self.static_obstacles))],
                                          dtype=np.float32)
        return {
            'agent': agent_state,
            'dyn_obs': dynamic_obstacle_states,
            'sta_obs': static_obstacle_states
        }
    
    def render(self, mode='human'):
        if not hasattr(self, 'fig'):
            self.fig, self.ax = plt.subplots()
            self.ax.set_xlim(0, self.grid_size)
            self.ax.set_ylim(0, self.grid_size)
            # Initializing markers with initial positions
            self.agent_marker, = self.ax.plot(self.agent_pos[0], self.agent_pos[1], 'go', markersize=10)
            self.goal_marker, = self.ax.plot(self.goal_pos[0], self.goal_pos[1], 'rx', markersize=10)
            self.dynamic_markers = [self.ax.plot(ob['current'][0], ob['current'][1], 'mo', markersize=5)[0] for ob in self.dynamic_obstacles]
            self.static_markers = [self.ax.plot(ob[0], ob[1], 'bs', markersize=5)[0] for ob in self.static_obstacles]
        else:
            # Ensuring All positions are sequences
            self.agent_marker.set_data([self.agent_pos[0]], [self.agent_pos[1]])
            self.goal_marker.set_data([self.goal_pos[0]], [self.goal_pos[1]])
            for marker, obstacle in zip(self.dynamic_markers, self.dynamic_obstacles):
                marker.set_data([obstacle['current'][0]], [obstacle['current'][1]])
            for marker, obstacle in zip(self.static_markers, self.static_obstacles):
                marker.set_data([obstacle[0]], [obstacle[1]])
        
        clear_output(wait=True)
        display(self.fig)
        plt.pause(0.01)  # A short pause to update the plot
    
    def reset(self, **kwargs):
        if 'seed' in kwargs:
            np.random.seed(kwargs['seed'])

        self.agent_pos = np.random.uniform(0, self.grid_size, size=2).round(2)
        self.goal_pos = np.random.uniform(0, self.grid_size, size=2).round(2)
        self.dynamic_obstacles = self._init_dynamic_obstacles()
        self.static_obstacles = self._init_static_obstacles()
        self.current_step = 0
        return self._get_obs(), {}

In [ ]:
from stable_baselines3.common.callbacks import BaseCallback

class RenderCallback(BaseCallback):
    def __init__(self, env, render_freq=1, verbose=0):
        super(RenderCallback, self).__init__(verbose)
        self.env = env
        self.render_freq = render_freq

    def _on_step(self):
        if self.n_calls % self.render_freq == 0:
            self.env.render()
        return True

In [ ]:
env = DGSN_Env()
obs = env.reset()

In [ ]:
log_path = os.path.join('Train','Logs')

In [ ]:
env.action_space

In [ ]:
env.observation_space

In [ ]:
env.observation_space.sample()

In [ ]:
render_callback = RenderCallback(env, render_freq=1)

In [ ]:
model = PPO("MultiInputPolicy", env, verbose=1, tensorboard_log=log_path, device='cuda', n_epochs=100)

In [ ]:
model.learn(total_timesteps=10000, callback=render_callback)